<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/B/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!nvidia-smi

Sun May 17 07:50:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir temp
#!cp '/content/drive/My Drive/Eva_DataSet.zip' temp
!cp '/content/drive/My Drive/sample10.zip' temp

In [0]:
destDataDir = '/content/temp/'
path_to_zip_file = '/content/temp/sample10.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
!mkdir data

In [0]:
!mv '/content/temp/sample10k/depth' '/content/data/'
!mv '/content/temp/sample10k/fgbg' '/content/data/'
!mv '/content/temp/sample10k/mask' '/content/data/'

In [11]:
!pip install kornia
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision.transforms import transforms
from matplotlib import pyplot as plt
import torchvision
from torch import nn
import os
import torch
from kornia.losses import SSIM

root = "data"


     |████████████████████████████████| 163kB 11.7MB/s 


In [0]:
class ImageDataSet(Dataset):
    def __init__(self, rt):
        self.root =rt
        self.transfrm_fgbg = transforms.Compose([transforms.Resize((200,200)),
        transforms.ToTensor(),transforms.Normalize((0.5314841953145771, 0.5454782214818802, 0.4948384681175141),(0.2673097911607354, 0.24659251689798323, 0.2906037084309324)),])
        
        self.transfrm_depth  = transforms.Compose([transforms.Resize((200,200)),transforms.ToTensor(),
                                                  transforms.Normalize(0.59442,0.251731),])
        
        self.transfrm_mask = transforms.Compose([transforms.Resize((200,200)),transforms.ToTensor(),
                                                 transforms.Normalize(0.02392,0.145103),])
        #put transforms here
        self.fgbg = os.listdir('data/fgbg/')
    def __len__(self):
        return len(self.fgbg)
    def __getitem__(self,index):
        img = self.fgbg[index]
        fgbg_img = self.transfrm_fgbg(Image.open(self.root + '/fgbg/'+img))
        msk_img  = self.transfrm_mask(Image.open(self.root + '/mask/'+img))
        depth_img= self.transfrm_depth(Image.open(self.root + '/depth/'+img))
        return {'image':fgbg_img, 'mask':msk_img,'depth':depth_img}


In [0]:
def FetchImage(root,index):
    imglst = os.listdir('data/fgbg/')
    img =imglst[index]
    #imgmsk = 'mask' + img[3:] 
    fgbg = Image.open(root + '/fgbg/'+img)
    msk = Image.open(root + '/mask/'+img)
    depth = Image.open(root + '/depth/'+img)
    return {'image':fgbg, 'mask':msk,'depth':depth}

In [0]:
class ConvGen(nn.Module):
  def __init__(self):
      super(ConvGen,self).__init__()

      self.convblock1 = nn.Sequential(
          nn.Conv2d(3, 32, 3, stride =1, padding =1, bias = False),
          nn.BatchNorm2d(32),
          nn.ReLU()
          )
      self.convblock2 = nn.Sequential(
          nn.Conv2d(32,64,3,stride=1, padding=1, bias=False),
          nn.Conv2d(64,128,1,stride=1,padding=0,bias=False),
          nn.BatchNorm2d(128),
          nn.ReLU()
          )
      self.convblock3 = nn.Sequential(
          nn.Conv2d(128,256,3,stride=1,padding=1,bias=False),
          nn.BatchNorm2d(256),
          nn.ReLU()
          )
      self.convblock4 = nn.Sequential(
          nn.Conv2d(256,1,3,stride=1,padding=1,bias= False),
          )
  def forward(self,sample):
     # print(sample.shape)
      f = sample['image'] 
      f = self.convblock2(self.convblock1(f))
      f = self.convblock4(self.convblock3(f))
      return f

In [0]:
trainset = ImageDataSet(root)
trainloader = DataLoader(trainset, batch_size = 16, shuffle =True, pin_memory=True)

In [52]:
criterion = nn.BCEWithLogitsLoss() #SSIM( 3, reduction ="mean")
model = ConvGen()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device=device)

# from torchsummary import summary
# summary(model, (2,3,200,200))

ConvGen(
  (convblock1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (convblock3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock4): Sequential(
    (0): Conv2d(256, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
)

In [0]:
optim = torch.optim.SGD(model.parameters(),lr= 0.01, momentum=0.9, weight_decay=1e-5)

In [0]:
def train(model,criterion,device,trainloader,optimizer,epochs):
    model.train()
    for i, data in enumerate(trainloader):
      data["image"] = data["image"].to(device)
      data["mask"] = data["mask"].to(device)
      #data["depth"] = data["depth"].to(device)

      optimizer.zero_grad()
      output = model(data)
      loss =criterion(output,data["mask"])
      loss.backward()
      optimizer.step()
      if i %500 == 0:
          print('Train Epoch: {}  [{}/{}  ({:.0f}%)]\tLoss:{:.6f}'.format(
              epochs,i*len(data),len(trainloader.dataset),
              100.*i/len(trainloader),loss.item()))
          print('Batch ID: ',i)


In [0]:
def test(model,criterion, device,testloader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.noo_grad():
        for data in testloader:
            data['image'] = data['image'].to(device)
            data['mask'] = data['mask'].to(device)
            output = model(data)

            test_loss += criterion(output,data['mask'],reduction='sum').item()
            pred = output.argmax(dim=1, keepdim =True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(testloader.dataset)


In [58]:

from torch.optim.lr_scheduler import StepLR

device = torch.device('cuda:0')
model = model.to(device)
scheduler = StepLR(optim, step_size =1, gamma=0.01)
for epoch in range(1, 10):
    train(model, criterion, device, trainloader,optim, epoch)
    print('epochs',epoch)
    scheduler.step()


Train Epoch: 1  [0/10000  (0%)]	Loss:-2460023521280.000000
Batch ID:  0
Train Epoch: 1  [150/10000  (8%)]	Loss:-2630079217664.000000
Batch ID:  50
Train Epoch: 1  [300/10000  (16%)]	Loss:-2332720627712.000000
Batch ID:  100
Train Epoch: 1  [450/10000  (24%)]	Loss:-3119403499520.000000
Batch ID:  150
Train Epoch: 1  [600/10000  (32%)]	Loss:-2555761917952.000000
Batch ID:  200
Train Epoch: 1  [750/10000  (40%)]	Loss:-2200840699904.000000
Batch ID:  250
Train Epoch: 1  [900/10000  (48%)]	Loss:-2514795888640.000000
Batch ID:  300
Train Epoch: 1  [1050/10000  (56%)]	Loss:-2565617221632.000000
Batch ID:  350
Train Epoch: 1  [1200/10000  (64%)]	Loss:-2716699983872.000000
Batch ID:  400
Train Epoch: 1  [1350/10000  (72%)]	Loss:-2486764568576.000000
Batch ID:  450
Train Epoch: 1  [1500/10000  (80%)]	Loss:-1980303933440.000000
Batch ID:  500
Train Epoch: 1  [1650/10000  (88%)]	Loss:-2605633503232.000000
Batch ID:  550
Train Epoch: 1  [1800/10000  (96%)]	Loss:-2623504908288.000000
Batch ID:  600


In [70]:
import numpy as np
model.eval()
x = np.clip(np.asarray(Image.open( '/content/1.jpg' ), dtype=float) / 255, 0, 1)
#x= x.to(device)
mg = model(x)


IndexError: ignored